# RAG Menggunakan Perkhidmatan Ejen Azure AI & Kernel Semantik

Kod ini menunjukkan cara untuk mencipta dan mengurus ejen Azure AI untuk penjanaan yang dipertingkatkan dengan pengambilan (RAG) menggunakan `Perkhidmatan Ejen Azure AI` dan `Kernel Semantik`. Ejen ini memproses pertanyaan pengguna berdasarkan konteks yang diperoleh dan memberikan respons yang tepat sesuai.


Pembaikan Versi SQLite  
Jika anda menghadapi ralat:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

Nyahkomen blok kod ini di permulaan buku nota anda:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### Mengimport Pakej
Kod berikut mengimport pakej yang diperlukan:


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# Penjanaan Augmentasi Pengambilan dengan Semantic Kernel & Azure AI Agent Service

Contoh ini menunjukkan cara menggunakan **Azure AI Agent Service** untuk melaksanakan **Retrieval-Augmented Generation (RAG)** dengan menggabungkan model bahasa dengan konteks khusus domain daripada dokumen yang dimuat naik.

### Cara Ia Berfungsi

1. **Muat Naik Dokumen**: Fail markdown (document.md) yang mengandungi maklumat (polisi insurans perjalanan Contoso) dimuat naik ke perkhidmatan ejen.

2. **Penciptaan Storan Vektor**: Dokumen diindeks ke dalam storan vektor untuk membolehkan carian semantik ke atas kandungannya.

3. **Konfigurasi Ejen**: Ejen diwujudkan menggunakan model `gpt-4o` dengan arahan ketat berikut:
   - Hanya menjawab soalan berdasarkan kandungan yang diambil daripada dokumen.
   - Menolak untuk menjawab jika soalan berada di luar skop.

4. **Integrasi Alat Carian Fail**: `FileSearchTool` didaftarkan dengan ejen, membolehkan model mencari dan mengambil petikan yang relevan daripada dokumen yang diindeks semasa inferens.

5. **Interaksi Pengguna**: Pengguna boleh mengemukakan soalan. Jika maklumat yang relevan dijumpai dalam dokumen, ejen menghasilkan jawapan yang berasaskan fakta.  
   Jika tidak, ejen secara eksplisit menjawab bahawa dokumen tidak mengandungi maklumat yang mencukupi.


### Fungsi Utama



Pastikan untuk menjalankan `az login` terlebih dahulu menggunakan Azure CLI supaya konteks pengesahan yang betul disediakan semasa menggunakan `DefaultAzureCredential`. Perkhidmatan Azure AI Agent tidak menggunakan kunci API.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

Baik, sila berikan fail markdown yang ingin diterjemahkan.



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk memastikan ketepatan, sila ambil perhatian bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang berwibawa. Untuk maklumat yang kritikal, terjemahan manusia profesional adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
